In [1]:
## usar python 3.8.8
import findspark
findspark.init()

In [2]:
# Importando SparkSession para criar uma sessão do Spark
from pyspark.sql import SparkSession

# Importando funções e tipos de dados SparkSQL
from pyspark.sql import functions as f
from pyspark.sql.types import *

# Importando módulos Spark MLlib
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import Imputer, StandardScaler, VectorAssembler
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder



# Importando SparkContext e SparkConf
from pyspark import SparkContext, SparkConf


In [3]:

# Criando uma nova sessão do Spark
# Spark entry point
spark = SparkSession \
    .builder \
    .appName("modeling-pkdd99-xpeMBA") \
    .getOrCreate()

spark.version

'3.0.0'

In [4]:

def read_df_csv(tabela=str):
    """
    Função para as bases de dados onde retorna no print o 'shape', um breve 'show' e o Scheema das variáveis.
    :param entidade_name: string que referencie o nome da tabela que complete o caminho './dados_originais/{tabela}.csv'. 
    tabela pode ser => trans, account, card, client, disp, district, loan, order 
    :return: DataFrame em pyspark
    """
    path ="C:\\Users\\renat\\Documents\\00_MBA\\PROJETO_APLICADO\\ML-predict-loan-MBA-applied-project\\dados_modelagem"
    df = spark.read.csv(path = f'{path}/{tabela}.csv', header='True',inferSchema='True', sep=',')
    print('\n','A base de dados possui:',df.count(), 'linhas', 'e', len(df.columns), 'colunas', '\n')
    print(df.show(5))
    print(df.printSchema())
    return(df)

# Base de entrada

In [5]:
df_model = read_df_csv('df_model')


 A base de dados possui: 827 linhas e 26 colunas 

+----------+-------+---------+---------------+----------------+--------+----------+---------+-------+----------+------+--------+--------+------+----------+------------------+-------+---------+--------+-------+-----------------+------------------+--------+--------+-----+------------+
|account_id|disp_id|client_id|account_id_acct|district_id_bank|stmt_frq| date_acct|type_disp|loan_id| date_loan|amount|duration|payments|status|date_birth|district_id_client|card_id|type_card|    min1|   max1|            mean1|             mean6|response|has_card|idade|days_between|
+----------+-------+---------+---------------+----------------+--------+----------+---------+-------+----------+------+--------+--------+------+----------+------------------+-------+---------+--------+-------+-----------------+------------------+--------+--------+-----+------------+
|     10351|  12430|    12738|          10351|              23| monthly|1995-05-04|    owner|   

In [6]:
df_model.take(1)

[Row(account_id=10351, disp_id=12430, client_id=12738, account_id_acct=10351, district_id_bank=23, stmt_frq='monthly', date_acct='1995-05-04', type_disp='owner', loan_id=7115, date_loan='1997-03-04', amount=88704, duration=48, payments=1848.0, status='C', date_birth='1960-10-29', district_id_client=23, card_id=None, type_card=None, min1=11853.6, max1=9953.6, mean1=18891.7448, mean6=19977.38913043479, response=1, has_card=0, idade=36, days_between=670)]

Definindo as variáveis em explicativas e resposta

In [7]:
df_model2 = df_model.select('amount', 'duration', 'payments', 'min1', 'max1', 'mean1', 'mean6', 'has_card', 'idade', 'days_between', 'response')

df_model2 =  df_model2.withColumnRenamed('response', 'label')

cols = df_model2.columns[:-1]

cols

['amount',
 'duration',
 'payments',
 'min1',
 'max1',
 'mean1',
 'mean6',
 'has_card',
 'idade',
 'days_between']

In [8]:
df_model2.printSchema()

root
 |-- amount: integer (nullable = true)
 |-- duration: integer (nullable = true)
 |-- payments: double (nullable = true)
 |-- min1: double (nullable = true)
 |-- max1: double (nullable = true)
 |-- mean1: double (nullable = true)
 |-- mean6: double (nullable = true)
 |-- has_card: integer (nullable = true)
 |-- idade: integer (nullable = true)
 |-- days_between: integer (nullable = true)
 |-- label: integer (nullable = true)



In [9]:
# contar valores ausentes por coluna
missing_counts = df_model2.select([f.sum(f.col(c).isNull().cast("int")).alias(c) for c in df_model2.columns])

# mostrar o resultado
missing_counts.show()


+------+--------+--------+----+----+-----+-----+--------+-----+------------+-----+
|amount|duration|payments|min1|max1|mean1|mean6|has_card|idade|days_between|label|
+------+--------+--------+----+----+-----+-----+--------+-----+------------+-----+
|     0|       0|       0|   0|   0|    0|   98|       0|    0|           0|    0|
+------+--------+--------+----+----+-----+-----+--------+-----+------------+-----+



In [10]:
df_model2.columns

['amount',
 'duration',
 'payments',
 'min1',
 'max1',
 'mean1',
 'mean6',
 'has_card',
 'idade',
 'days_between',
 'label']

In [11]:
df_model2.select('mean6').summary('mean').show()

+-------+-----------------+
|summary|            mean6|
+-------+-----------------+
|   mean|40452.95037533935|
+-------+-----------------+



In [12]:
df_model2 = df_model2.fillna({'mean6': 40452.95})

Criar um VectorAssembler para transformar as variáveis explicativas em uma única coluna de vetores

In [13]:
# Define o VectorAssembler para unir as colunas em uma única coluna vetorizada
assembler = VectorAssembler(
    inputCols=df_model2.columns[:-1],
    outputCol="features"
)

 MinMaxScaler ou o StandardScaler para dimensionar os dados para que características como salário, idade e renda contribuam igualmente para a análise contribuam igualmente para a análise. Ao dimensionar os dados, podemos garantir que cada recurso tenha um impacto igual no desempenho do modelo, e o modelo pode fazer previsões mais precisas.

O conceito de normalização é implementado em Python usando MinMaxScaler e o conceito de padronização é implementado usando StandardScaler.
 MinMaxScaler dimensiona os dados para um intervalo fixo, normalmente entre 0 e 1. Por outro lado, StandardScaler redimensiona os dados para que tenham uma média de 0 e um desvio padrão de 1. Isso resulta em uma distribuição com média zero e variação de unidade. 
 
 https://vitalflux.com/minmaxscaler-standardscaler-python-examples/

Criando o StandardScaler

In [14]:
# Define o StandardScaler para normalizar as variáveis
scaler = StandardScaler(
    inputCol="features",
    outputCol="scaledFeatures",
    withStd=True,
    withMean=False
)

Definindo o pipeline para unir as etapas de pré-processamento

In [15]:
# Define o Pipeline com as etapas de pré-processamento e o modelo

pipeline = Pipeline(stages=[assembler, scaler])

In [16]:
pipelineModel = pipeline.fit(df_model2)

In [17]:
df_transformed = pipelineModel.transform(df_model2)

In [18]:
df_transformed.show()

+------+--------+--------+--------+-------+------------------+------------------+--------+-----+------------+-----+--------------------+--------------------+
|amount|duration|payments|    min1|   max1|             mean1|             mean6|has_card|idade|days_between|label|            features|      scaledFeatures|
+------+--------+--------+--------+-------+------------------+------------------+--------+-----+------------+-----+--------------------+--------------------+
| 88704|      48|  1848.0| 11853.6| 9953.6|        18891.7448| 19977.38913043479|       0|   36|         670|    1|[88704.0,48.0,184...|[0.77418650865161...|
| 88704|      48|  1848.0| 11853.6| 9953.6|        18891.7448| 19977.38913043479|       0|   39|         670|    1|[88704.0,48.0,184...|[0.77418650865161...|
| 54396|      36|  1511.0| 18196.0| 8296.0|       27385.44375|             800.0|       0|   57|         191|    1|[54396.0,36.0,151...|[0.47475479487524...|
|143904|      24|  5996.0|100282.7|96774.9| 49212.63

Divide o dataset em conjuntos de treinamento e teste

In [19]:
(trainData, testData) = df_transformed.randomSplit([0.7, 0.3], seed=12345)

Definindo o modelo de Regressão Logística

No MLLib, a Regressão Logística não possui um método de cálculo de feature importance embutido. No entanto, é possível calcular a importância das features de forma aproximada usando o método "L1 Regularization" (também conhecido como "Lasso regularization").

Essa técnica de regularização penaliza os coeficientes das features que não são relevantes para a predição, forçando-os a terem valor zero. Dessa forma, as features com coeficientes não-nulos são consideradas mais importantes.

Para realizar esse método no MLLib, basta usar o parâmetro "elasticNetParam" da função LogisticRegression, que controla a proporção de regularização L1 e L2. Ao definir o valor de "elasticNetParam" como 1 (que significa 100% de regularização L1), a regressão logística irá utilizar apenas L1 regularization, o que resultará em alguns coeficientes com valor zero.

In [20]:
lr = LogisticRegression(featuresCol = 'scaledFeatures', labelCol = 'label', elasticNetParam=1)

Define os parâmetros a serem testados

In [21]:
# 
#paramGrid = ParamGridBuilder() \
#    .addGrid(lr.regParam, [0.1, 0.01]) \
#    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0]) \
#    .build()

Definindo o avaliador para a validação cruzada

In [22]:

evaluator = BinaryClassificationEvaluator(
    labelCol='label',
    rawPredictionCol="rawPrediction",
    metricName="areaUnderROC"
)

Definindo a validação cruzada com 5 folds

In [23]:

# 
#crossval = CrossValidator(
#    estimator=lr,
#    estimatorParamMaps=paramGrid,
#    evaluator=evaluator,
#    numFolds=5
#)

Ajusta o modelo com a validação cruzada

In [24]:
# 
#modelo = crossval.fit(trainData)

In [25]:
# 
modelo = lr.fit(trainData)

In [26]:
coefficients = modelo.coefficients.toArray()

In [27]:
coefficients

array([-0.44140367,  0.19925389, -0.19457131,  0.46574029, -0.36123858,
        0.67418071, -0.28387973,  0.39524176, -0.11496371,  0.31963519])

In [28]:
# obtém as features mais importantes (aquelas com coeficiente não nulo)
important_features = [i for i, coef in enumerate(coefficients) if coef != 0]

In [29]:
feature_importance = list(zip(assembler.getInputCols(), modelo.coefficients.toArray()))

In [30]:
feature_importance

[('amount', -0.44140366598496533),
 ('duration', 0.1992538902862431),
 ('payments', -0.1945713091696159),
 ('min1', 0.46574029122558336),
 ('max1', -0.36123857645703733),
 ('mean1', 0.6741807087636067),
 ('mean6', -0.2838797275156386),
 ('has_card', 0.3952417574842975),
 ('idade', -0.11496370821418399),
 ('days_between', 0.31963519439202237)]

In [31]:
# assuming `model` is your trained LogisticRegressionModel object, and `trainData` is your training data DataFrame
predictions_train = modelo.transform(trainData)
auc_train = evaluator.evaluate(predictions_train)
print("AUC on training data = %g" % auc_train)

AUC on training data = 0.752386


Avalia o modelo com os dados de teste

In [32]:
# assuming `model` is your trained LogisticRegressionModel object, and `trainData` is your training data DataFrame
predictions_teste = modelo.transform(testData)
auc_test = evaluator.evaluate(predictions_teste)
print("AUC on training data = %g" % auc_test)

AUC on training data = 0.734202


In [33]:
predictions_teste.printSchema()

root
 |-- amount: integer (nullable = true)
 |-- duration: integer (nullable = true)
 |-- payments: double (nullable = true)
 |-- min1: double (nullable = true)
 |-- max1: double (nullable = true)
 |-- mean1: double (nullable = true)
 |-- mean6: double (nullable = false)
 |-- has_card: integer (nullable = true)
 |-- idade: integer (nullable = true)
 |-- days_between: integer (nullable = true)
 |-- label: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- scaledFeatures: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [34]:
predictions_teste.select('probability').show(truncate=False)

+-----------------------------------------+
|probability                              |
+-----------------------------------------+
|[0.057585893661319866,0.9424141063386803]|
|[0.024207806939383646,0.9757921930606164]|
|[0.017197850174292668,0.9828021498257073]|
|[0.038465008801542966,0.961534991198457] |
|[0.07434340271245267,0.9256565972875472] |
|[0.04062857484270776,0.9593714251572923] |
|[0.07861198400648306,0.9213880159935169] |
|[0.0689411472811062,0.9310588527188939]  |
|[0.09163813490888457,0.9083618650911155] |
|[0.09927361069329263,0.9007263893067073] |
|[0.03748266207665693,0.9625173379233432] |
|[0.04463708220051664,0.9553629177994833] |
|[0.0584679374786745,0.9415320625213255]  |
|[0.016960692927537,0.983039307072463]    |
|[0.03546721660805609,0.9645327833919438] |
|[0.04016570078116782,0.9598342992188322] |
|[0.011957337745994341,0.9880426622540057]|
|[0.0411236486029402,0.9588763513970598]  |
|[0.04688460051171686,0.9531153994882832] |
|[0.13646218129213358,0.86353781

Observando as métricas do modelo

In [35]:
# Importa as métricas de avaliação
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Cria o avaliador com a métrica de área sob a curva PR (precisão e recall)
evaluator = BinaryClassificationEvaluator(metricName="areaUnderPR")

# Usa o modelo para fazer previsões na base de teste
predictions_teste = modelo.transform(testData)

# Calcula a métrica de precisão
precision = evaluator.evaluate(predictions_teste)

# Calcula a métrica de recall
evaluator = BinaryClassificationEvaluator(metricName="areaUnderROC")
recall = evaluator.evaluate(predictions_teste)


print("Precision = %g" % precision)
print("Recall = %g" % recall)

Precision = 0.949816
Recall = 0.734202
